In [1]:
import numpy as np

## Q5: Einstein summation

einsum is a powerful (but often painful) numpy thing:
- https://numpy.org/doc/stable/reference/generated/numpy.einsum.html
- https://stackoverflow.com/questions/26089893/understanding-numpys-einsum

Take 2 vectors A and B. Write the einsum equivalent of inner, outer, sum, and mul function.

In [2]:
A = np.random.rand(10)
B = np.random.rand(10)

### inner ###
print(f'inner  :  einsum = {np.einsum("i,i", A, B)}, numpy = {np.inner(A, B)}\n') # A_i B_i 

### outer ###
print(f'outer  :  einsum - numpy = \n{np.einsum("i,j", A, B) - np.outer(A, B)}\n') # A_i B_j = C_ij, should be equal to 0 matrix. 

### sum ### 
print(f'sum A  :  numpy = {np.sum(A)}, einsum = {np.einsum("i->", A)}')
print(f'sum B  :  numpy = {np.sum(B)}, einsum = {np.einsum("i->", B)}\n')

### matmul ###
A = np.random.rand(10, 10)
B = np.random.rand(10, 10)

print(f'matmul  :  numpy - einsum = \n{np.matmul(A, B) - np.einsum("ij,jk->ik", A, B)}\n')

inner  :  einsum = 1.9161957081950587, numpy = 1.9161957081950591

outer  :  einsum - numpy = 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

sum A  :  numpy = 5.043069328174349, einsum = 5.043069328174347
sum B  :  numpy = 4.097499860307841, einsum = 4.097499860307841

matmul  :  numpy - einsum = 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]



## Q6: Conway's Game of Life

**Exercise**: Code up Conway's Game of Life using numpy 

The Game of Life is a cellular automaton devised by mathematician John Horton Conway in 1970. It is a zero-player game, meaning that its evolution is determined by its initial state, requiring no further input. One interacts with the Game of Life by creating an initial configuration and observing how it evolves. It is Turing complete and can simulate a universal constructor or any other Turing machine.

https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life

The Game of Life is *really* (really, really) cool. There are just four extremely simple rules, and these result in an immense richness of behaviour and complexity.

https://www.youtube.com/watch?v=C2vgICfQawE&t=221s&ab_channel=RationalAnimations

https://www.youtube.com/watch?v=jvSp6VHt_Pc&ab_channel=TheDevDoctor

Here some web apps to play:

https://conwaylife.com/

https://playgameoflife.com/

Some computational hints:

https://blog.datawrapper.de/game-of-life/

For instance, here is a Game-of-Life structure that sends a message at fixed intervals (that little spaceship leaving toward the bottom right)

![](https://blog.datawrapper.de/wp-content/uploads/2021/06/game-of-life-loop-cropped.gif)


In [16]:



import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt 
import matplotlib.animation as animation


### routine to update the state according to the rules ###
def update_state(frames, img, state, N):
    ### counting the living neighbors using np.roll
    live = (np.roll(state,  1, axis=0) +                        # center-north
            np.roll(state, -1, axis=0) +                        # center-south
            np.roll(state,  1, axis=1) +                        # center-east   
            np.roll(state, -1, axis=1) +                        # center-west
            np.roll(np.roll(state,  1, axis=0), 1, axis=1)  +   # north-east
            np.roll(np.roll(state,  1, axis=0), -1, axis=1) +   # north-west
            np.roll(np.roll(state, -1, axis=0), 1, axis=1)  +   # south-east
            np.roll(np.roll(state, -1, axis=0), -1, axis=1))    # south-west
    
    ### apply the rules
    new_state = np.where((state == 1) & ((live < 2) | (live > 3)), 0, state)    # dead by underpopulation and overpopulation
    new_state = np.where((state == 0) & (live == 3), 1, new_state)              # new live by reproduction 
    
    # update data for the image plot
    img.set_data(new_state)
    # update the state for the next iteration
    state[:] = new_state[:]
    return img


def play(N=40, n_step=50, case='random'):
    ### Intializing grid and starting cnfg ###
    state = np.zeros((N, N), dtype=int)

    if case=='random':
        ### Random initial state 
        state = np.random.randint(0, 2, size=(N,N))
    else:
        ### Personalized initial state ###
        state[1, 1:3] = [1, 1]
        state[2, 1:3] = [1, 1]
        state[1, 15:17] = [1, 1]
        state[2, 15:17] = [1, 1]  # these are 2x2 block that live forever

        state[5, 10] = 1
        state[6, 10] = 1
        state[7, 10] = 1
        state[7, 22] = 1
        state[8, 22] = 1
        state[9, 22] = 1
        state[13, 10:13] = [1, 1, 1]
        state[14,  9:12] = [1, 1, 1]  # these are different types of oscillators

        state[7, 19]    = 1
        state[8, 20:22] = [1, 1]
        state[9, 19:21] = [1, 1]

    ### animation settings ###
    fig, ax = plt.subplots()
    color_map = mpl.colors.ListedColormap(["white", "black"])
    ims = ax.imshow(state, interpolation='nearest', cmap=color_map)
    ax.set_yticks(np.arange(-0.5, N-1, 1))
    ax.set_xticks(np.arange(-0.5, N-1, 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(color='grey', linestyle='-', linewidth=1)

    ani = animation.FuncAnimation(fig        = fig, 
                                    func       = update_state, 
                                    fargs      = (ims, state, N,),
                                    frames     = n_step,
                                    interval   = 800,
                                    )

    
    plt.close(fig)
    
    return HTML(ani.to_jshtml())




Examples of application

In [17]:
# Example 1: random initial state
play(case='random')




In [18]:
# Example 2: personalized initial state
play(N=25, n_step=50, case='custom')